In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
import pyxem
import diffpy
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator
from pyxem.utils import indexation_utils as iutls
from pyxem.utils import plotting_utils as putls
from pyxem.utils import polar_transform_utils as ptutls
from pyxem.utils import expt_utils as eutls
from orix.quaternion import Orientation, Rotation, symmetry
from orix.vector import AxAngle, Vector3d
import random

In [4]:
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
resolution =1  # maximum angle in degrees between nearest templates. Pretty rough grid for speed.
grid_cub = get_beam_directions_grid("cubic", resolution, mesh="spherified_cube_edge")
grid_hex = get_beam_directions_grid("hexagonal", resolution, mesh="spherified_cube_edge")


# Parameters necessary for simulating a template library
diffraction_calibration = 0.01
# half size of the images
half_shape = (256//2, 256//2)
# maximum radius in reciprocal space to calculate spot intensities for
reciprocal_radius = np.sqrt(half_shape[0]**2 + half_shape[1]**2)*diffraction_calibration

#structures
structure_matrix = diffpy.structure.loadStructure("Data/Au_mp-81_conventional_standard.cif")
structure_matrix2 = diffpy.structure.loadStructure("Data/Ti_mp-72_conventional_standard.cif")
# structure_matrix = diffpy.structure.loadStructure("Data/Pd_mp-2_conventional_standard.cif")

# "The microscope"
diff_gen = DiffractionGenerator(accelerating_voltage=200,
                                precession_angle=1,
                                scattering_params=None,
                                shape_factor_model="linear",
                                minimum_intensity=0.1,
                                )

lib_gen = DiffractionLibraryGenerator(diff_gen)

# Generating a library
# "Library of structures and orientations"
library_phases_all = StructureLibrary(["Au", "Ti"], [structure_matrix, structure_matrix2], [grid_cub, grid_hex])
# Calculate the actual library
# library_phases_corners = StructureLibrary(["Au"], [structure_matrix], [corners])
# diff_lib = lib_gen.get_diffraction_library(library_phases_corners,
#                                            calibration=diffraction_calibration,
#                                            reciprocal_radius=reciprocal_radius,
#                                            half_shape=half_shape,
#                                            with_direct_beam=False,
#                                            max_excitation_error=0.08)
diff_lib_full = lib_gen.get_diffraction_library(library_phases_all,
                                           calibration=diffraction_calibration,
                                           reciprocal_radius=reciprocal_radius,
                                           half_shape=half_shape,
                                           with_direct_beam=False,
                                           max_excitation_error=0.08)

In [35]:
corners = np.array([[ 0.,  0., 90.],
       [ 0.,  45., 90.],
       [ 0.,  54.7, 45.]])

In [5]:
from pyxem.utils.plotting_utils import get_template_cartesian_coordinates 
from diffsims.pattern.detector_functions import add_shot_and_point_spread
import random

def pointplotter(simulations,inplane,template,sigma):
    pattern = hs.signals.Signal2D(np.zeros((256,256)))
    pattern.set_signal_type(signal_type="electron_diffraction")
    x, y, intensities = get_template_cartesian_coordinates(simulations[template],in_plane_angle=inplane,window_size=(255,255),center=(256//2,256//2))
    for i in range(x.size):
        pattern.isig[x[i],y[i]]=intensities[i]
    pattern = add_shot_and_point_spread(pattern.T, sigma, shot_noise=False)
    pattern = hs.signals.Signal2D(pattern)
    return pattern
def grab_random_pattern(simulations):
    inplane = random.randint(0,360)
    template = random.randint(0,simulations.size-1)
    pattern = pointplotter(simulations,inplane,template,5)
    return pattern, inplane, template
def build_random_tilts(diff_lib,n):
    orientations = diff_lib['orientations']
    simulations = diff_lib['simulations']
    pattern_set = hs.signals.Signal2D(np.zeros((n,1,256,256)))
    pattern_set.set_signal_type(signal_type="electron_diffraction")
    eulers = np.zeros((n,3))
    template_indexs = np.zeros((1,n))
    for i in range(n):
        pattern, inplane, template = grab_random_pattern(simulations)
        euler = orientations[template].copy()
        euler[0] = euler[0] + inplane
        pattern_set.inav[0,i] = pattern
        eulers[i,:] = euler
        template_indexs[0,i] = template
    return pattern_set,eulers,template_indexs

In [6]:
def rotate_sims_original(random_tilts,eulers,rotation,num_rot):
#     quats = Orientation.from_euler(np.radians(eulers),symmetry=Oh)
#     rotated_quats = (quats * Orientation((1,0,np.radians(5),0))).set_symmetry(Oh)
#     rotated_eulers = np.degrees(rotated_quats.to_euler())
#     rotated_eulers = np.degrees(qu2eu(rotated_quats.data))
    shape = random_tilts.data.shape
    pattern_set = hs.signals.Signal2D(np.zeros((shape[0],shape[1] + num_rot,shape[2],shape[3])))
    pattern_set.inav[0,:] = random_tilts.data[:,0,:,:]
    all_eulers = np.zeros((100,num_rot + 1,3))
    all_eulers[:,0,:] = eulers
    for i in range(0, num_rot):
        rotated_eulers = np.degrees(om2eu(np.matmul(eu2om(np.radians(all_eulers[:,i,:])) ,rotation)))
        all_eulers[:,i+1,:] = rotated_eulers
    #     rotated_eulers = in_euler_fundamental_region(np.radians(rotated_eulers))
        diff_gen = DiffractionGenerator(accelerating_voltage=200,
                                    precession_angle=1,
                                    scattering_params=None,
                                    shape_factor_model="linear",
                                    minimum_intensity=0.1,
                                    )

        lib_gen = DiffractionLibraryGenerator(diff_gen)
        # Generating a library
        # "Library of structures and orientations"
        library_phases = StructureLibrary(["phase"], [structure_matrix], [rotated_eulers])
        diff_lib = lib_gen.get_diffraction_library(library_phases,
                                               calibration=diffraction_calibration,
                                               reciprocal_radius=reciprocal_radius,
                                               half_shape=half_shape,
                                               with_direct_beam=False,
                                               max_excitation_error=0.08)

        
            
        for j in range(eulers.shape[0]):        
            pattern = pointplotter(diff_lib['phase']['simulations'],0,j,5)
            pattern_set.inav[i+1,j] = pattern
            
    return(pattern_set,all_eulers)

In [7]:
def rotate_sims(random_tilts,eulers,rot_quat,num_rot,pg):
#     quats = Orientation.from_euler(np.radians(eulers),symmetry=Oh)
#     rotated_quats = (quats * Orientation((1,0,np.radians(5),0))).set_symmetry(Oh)
#     rotated_eulers = np.degrees(rotated_quats.to_euler())
#     rotated_eulers = np.degrees(qu2eu(rotated_quats.data))
    shape = random_tilts.data.shape
    pattern_set = hs.signals.Signal2D(np.zeros((shape[0],shape[1] + num_rot,shape[2],shape[3])))
    pattern_set.inav[0,:] = random_tilts.data[:,0,:,:]
    all_eulers = np.zeros((shape[0],num_rot + 1,3))
    all_eulers[:,0,:] = eulers
    for i in range(0, num_rot):
#         rotated_eulers = np.degrees(om2eu(np.matmul(eu2om(np.radians(all_eulers[:,i,:])) ,rotation)))
        quats = Orientation.from_euler(np.radians(all_eulers[:,i,:]),symmetry=pg)
        rotated_quats = (quats * rot_quat)
        rotated_quats.symmetry = pg
        rotated_eulers = np.degrees(rotated_quats.to_euler())
        
        all_eulers[:,i+1,:] = rotated_eulers
        diff_gen = DiffractionGenerator(accelerating_voltage=200,
                                    precession_angle=1,
                                    scattering_params=None,
                                    shape_factor_model="linear",
                                    minimum_intensity=0.1,
                                    )

        lib_gen = DiffractionLibraryGenerator(diff_gen)
        # Generating a library
        # "Library of structures and orientations"
        library_phases = StructureLibrary(["phase"], [structure_matrix], [rotated_eulers])
        diff_lib = lib_gen.get_diffraction_library(library_phases,
                                               calibration=diffraction_calibration,
                                               reciprocal_radius=reciprocal_radius,
                                               half_shape=half_shape,
                                               with_direct_beam=False,
                                               max_excitation_error=0.08)

        
            
        for j in range(eulers.shape[0]):        
            pattern = pointplotter(diff_lib['phase']['simulations'],0,j,5)
            pattern_set.inav[i+1,j] = pattern
            
    return(pattern_set,all_eulers)

In [8]:
def full_lib_thing(simulations,inplane):
    allpatterns = hs.signals.Signal2D(np.zeros((1,simulations.size,256,256)))
    allpatterns.set_signal_type(signal_type="electron_diffraction")
    pattern = hs.signals.Signal2D(np.zeros((256,256)))
    pattern.set_signal_type(signal_type="electron_diffraction")

    for i in range(simulations.size):
        pattern = pointplotter(simulations,inplane,i,5)
        allpatterns.inav[i,:] = pattern
        pattern = hs.signals.Signal2D(np.zeros((256,256)))
        
    allpatterns.set_diffraction_calibration(0.01)
    return allpatterns

In [19]:
simulations = diff_lib["Au"]["simulations"]
allpatterns = full_lib_thing(simulations,0)

In [20]:
allpatterns.plot()

In [6]:
data0  = hs.load('set_data0.hspy')
import pickle
pickle_off = open("set_eulers0.pickle", 'rb')
eulers0 = pickle.load(pickle_off)

EOFError: Ran out of input

In [11]:
data0, eulers0, template_indexs = build_random_tilts(diff_lib_full["Au"],200)
# data1, eulers1 = rotate_sims_original(data0,eulers0,rotmatxy,10)

In [13]:
axang12 = AxAngle.from_axes_angles((-1, 0, 0), np.radians(5))
g12 = Orientation.from_neo_euler(axang12, symmetry.Oh)
g12.map_into_symmetry_reduced_zone(symmetry)
print(g12)
print(g12.axis)
print(g12.axis.in_fundamental_sector(symmetry.Oh))

  0%|          | 0/48 [00:00<?, ?it/s]

Orientation (1,) m-3m
[[ 0.999  -0.0436  0.      0.    ]]
Vector3d (1,)
[[-1.  0.  0.]]
Vector3d (1,)
[[0. 0. 1.]]


In [12]:
from diffsims.libraries import diffraction_library

In [15]:
data0, eulers0, template_indexs = build_random_tilts(diff_lib_full['Au'],200)
data1, eulers1 = rotate_sims(data0,eulers0,g12,10,symmetry.Oh)

In [23]:
import pickle
filename = "set_eulers0.pickle"
pickling_on = open(filename,"wb")
pickle.dump(eulers0, pickling_on)
pickling_on.close()

In [55]:
data0.save('set_data0')

In [106]:
Orientation.from_matrix(rotmatxy).axis

Vector3d (1,)
[[0.7069 0.7069 0.0218]]

In [ ]:
Orientation.from_euler().axis

In [16]:
data1.plot()

In [ ]:
m12 = ~ori1 -  

In [8]:
testeuler1 = np.array([91.      ,   35.40869539, 80.05404644])
testeuler2 = np.array([93.      ,   38.83764774 ,76.0287308 ])

om1 = eu2om(np.radians(testeuler1))
om2 = eu2om(np.radians(testeuler2))

In [9]:
rotmat = np.matmul(np.transpose(om1),om2)

In [ ]:
axang12 = AxAngle.from_axes_angles((-1, -1, 0), np.radians(5))
g12 = Orientation.from_neo_euler(axang12, pg)
g2 = g12 * g1
g2.symmetry = pg
g2

In [155]:
data1.set_signal_type('electron_diffraction')
data1.set_diffraction_calibration(0.01)

In [137]:
data1.plot()

In [156]:
data1.inav[:,0].plot(cmap = 'viridis')

In [144]:
tiltors = Orientation.from_euler(np.radians(eulers1[0,:,:]), symmetry= symmetry.Oh)

In [157]:
fig0 = tiltors[0].plot_unit_cell(return_figure=True)
fig3 = tiltors[3].plot_unit_cell(return_figure=True)
fig6 = tiltors[6].plot_unit_cell(return_figure=True)
fig10 = tiltors[10].plot_unit_cell(return_figure=True)

In [158]:
fig0.savefig('UnitCell0.png',bbox_inches = 'tight', pad_inches = 0)
fig3.savefig('UnitCell3.png',bbox_inches = 'tight', pad_inches = 0)
fig6.savefig('UnitCell6.png',bbox_inches = 'tight', pad_inches = 0)
fig10.savefig('UnitCell10.png',bbox_inches = 'tight', pad_inches = 0)

In [79]:
data1.inav[:,50] = pointplotter(diff_lib["Ti"]["simulations"],30,300,5)

In [7]:
misori = np.array([-0.99927228, -0.0211923 , -0.01650285, -0.02708252])
rotmat = qu2om(misori)

array([[ 0.9979884 , -0.05342616,  0.03412956],
       [ 0.05482509,  0.99763486, -0.04145988],
       [-0.0318338 ,  0.04324763,  0.99855709]])

In [25]:
# data1 = data1 * 50

In [17]:
def template_log_func(x):
        return np.log10(x + 0.01)

In [18]:
frac_keep = 1
n_best = 20
n_keep = None

delta_r = 1
delta_theta = 0.5 
max_r = None
intensity_transform_function = template_log_func
direct_beam_position = None
normalize_image = True
normalize_templates = True

result, phasedict = iutls.index_dataset_with_template_rotation(data1,
                                                    diff_lib_full,
                                                    phases = ["Au"],
                                                    n_best = n_best,
                                                    frac_keep = frac_keep,
                                                    n_keep = n_keep,
                                                    delta_r = delta_r,
                                                    delta_theta = delta_theta,
                                                    max_r = None,
                                                    intensity_transform_function=intensity_transform_function,
                                                    normalize_images = normalize_image,
                                                    normalize_templates=normalize_templates,
                                                    target='gpu'
                                                    )

[########################################] | 100% Completed | 26.9s


In [83]:
phasedict

{0: 'Au', 1: 'Ti'}

In [100]:
data1.plot()

In [84]:
solution = result["orientation"]
corrscores = result['correlation'][:,:,0]

In [19]:
from orix.crystal_map import Phase
from orix.plot import IPFColorKeyTSL
from orix.quaternion import Orientation, symmetry
from orix.vector import Miller, Vector3d

plt.rcParams.update({
    "figure.figsize": (15, 10),
    "axes.grid": False,
#     "font.size": 20,
    "lines.markersize": 15,
})

In [29]:
def plot_matchedandtrue_Oh(num):
    
    
    oritrue = Orientation.from_euler(
        np.radians(eulers1[num,:,:]),
        symmetry=symmetry.Oh
    )
    orimatched = Orientation.from_euler(
    np.radians(result['orientation'][num,:,0,:]),
    symmetry=symmetry.Oh
    )
    ckey = IPFColorKeyTSL(oritrue.symmetry)
    rgbz = ckey.orientation2color(oritrue)
    # cmap = np.linspace(0, 1, oritrue.size)
    # v = Vector3d(((1, 0, 0), (0, 1, 0), (0, 0, 1)))
    v = Vector3d((0, 0, 1))
    oritrue.scatter("ipf", direction=v, c=green)
    orimatched.scatter("ipf", direction=v, c=red)
    

In [40]:
def plot_matchedandtrue_Oh(num):
    
    
    oritrue = Orientation.from_euler(
        np.radians(eulers1[num,:,:]),
        symmetry=symmetry.Oh
    )
    orimatched = Orientation.from_euler(
    np.radians(result['orientation'][num,:,0,:]),
    symmetry=symmetry.Oh
    )
    ckey = IPFColorKeyTSL(oritrue.symmetry)
    rgbz = ckey.orientation2color(oritrue)
    cmap = np.linspace(0, 1, oritrue.size)
    # v = Vector3d(((1, 0, 0), (0, 1, 0), (0, 0, 1)))
    v = Vector3d((0, 0, 1))
    fig = oritrue.scatter("ipf", direction=v, c='g', return_figure = True, label = 'Known')
    orimatched.scatter("ipf", direction=v, c='r', figure = fig, label = 'Matched', marker = 's')
    plt.legend()

In [12]:
def plot_matchedandtrue_D6h(num):
    
    
    oritrue = Orientation.from_euler(
        np.radians(eulers1[num,:,:]),
        symmetry=symmetry.D6h
    )
    orimatched = Orientation.from_euler(
    np.radians(result['orientation'][num,:,0,:]),
    symmetry=symmetry.D6h
    )
    ckey = IPFColorKeyTSL(oritrue.symmetry)
    rgbz = ckey.orientation2color(oritrue)
    cmap = np.linspace(0, 1, oritrue.size)
    v = Vector3d(((1, 0, 0), (0, 1, 0), (0, 0, 1)))
    oritrue.scatter("ipf", direction=v, c=cmap)
    orimatched.scatter("ipf", direction=v, c=cmap)

In [30]:
orimatched = Orientation.from_euler(
    np.radians(result['orientation'][6,:,0,:]),
    symmetry=symmetry.Oh
)

In [ ]:
orimatched.scatter()

In [147]:
ori = Orientation.from_euler(
    np.radians(eulers1[6,:,:]),
    symmetry=symmetry.Oh
)

In [74]:
plot_matchedandtrue_Oh(89)

In [148]:
np.degrees((ori - orimatched).to_euler())

array([[-117.37629495,    0.64573202,  116.61272679],
       [-128.11072225,    0.85030879,  127.2500142 ],
       [-143.21597255,    0.49545274,  142.60859118],
       [ -96.00234405,    0.3918114 ,   95.531186  ],
       [-132.2353125 ,    0.66954394,  131.43135781],
       [-137.87814497,    0.25713887,  137.39801596],
       [-148.51497217,    0.44441057,  148.02031093],
       [ -98.93647383,    0.28349625,   98.0992975 ],
       [-160.50756703,    0.55156262,  159.95041848],
       [-158.75750275,    0.25347986,  158.4420215 ],
       [-108.19542868,    0.16059081,  107.57528775]])

In [120]:
np.degrees((ori - orimatched).angle.data)

array([0.        , 0.3642247 , 0.30811811, 0.41799645, 0.50365635,
       0.8610643 , 0.6106717 , 0.53470235, 0.6092405 , 0.57420749,
       1.05712982])

In [140]:
from orix.vector import AxAngle

In [141]:
AxAngle1 = AxAngle.from_axes_angles((0,0,-1),np.pi)
orisomething = Orientation.from_neo_euler(AxAngle1)

In [142]:
orimatched2 = orisomething * orimatched

In [154]:
orimatched3 = orimatched2

In [153]:
orimatched2 = orimatched2.set_symmetry(symmetry.Oh)

/home/joseph/miniconda3/envs/pyxemmaster/lib/python3.9/site-packages/orix/quaternion/orientation.py:624: VisibleDeprecationWarning: Function `set_symmetry()` is deprecated and will be removed in version 0.9. Use `orix.quaternion.Orientation.map_into_symmetry_reduced_zone()` instead.
  since="0.8",


In [160]:
ori2 = ori.set_symmetry(symmetry.Oh)

/home/joseph/miniconda3/envs/pyxemmaster/lib/python3.9/site-packages/orix/quaternion/orientation.py:624: VisibleDeprecationWarning: Function `set_symmetry()` is deprecated and will be removed in version 0.9. Use `orix.quaternion.Orientation.map_into_symmetry_reduced_zone()` instead.
  since="0.8",


In [161]:
uvw = Miller(uvw=(1, 0, 0), phase=Phase(point_group=orimatched.symmetry))
uvw2 = ~orimatched3 * uvw
uvw3 = ~ori2 * uvw

In [164]:
ori2.scatter('rodrigues')

In [165]:
orimatched3.scatter('rodrigues')

In [163]:
fig = uvw2.scatter(return_figure = True,c=cmap, axes_labels=["x", "y", "z"], grid=True,hemisphere = 'both',s = 60)
# uvw3.scatter(figure = fig,c=cmap, axes_labels=["x", "y", "z"], grid=True,hemisphere = 'both', cmap = 'plasma', s = 60)

In [145]:
fig = orimatched.axis.scatter(return_figure = True,c=cmap, axes_labels=["x", "y", "z"], grid=True,hemisphere = 'both',s = 60)
ori.axis.scatter(figure = fig,c=cmap, axes_labels=["x", "y", "z"], grid=True,hemisphere = 'both', cmap = 'plasma', s = 60)

In [18]:
ckey = IPFColorKeyTSL(ori.symmetry)

In [19]:
rgbz = ckey.orientation2color(ori)
cmap = np.linspace(0, 1, ori.size)

In [ ]:
v = Vector3d(((1, 0, 0), (0, 1, 0), (0, 0, 1)))
ori.scatter("ipf", direction=v, c=cmap)

In [ ]:
vec = Vector3d(ori).in_fundamental_sector(symmetry.Oh)

cmap = np.linspace(0, 1, vec.size)
fig_kwargs = dict(figsize=(5, 5))
labels = ["x", "y", None]
vec.scatter(    
    hemisphere="upper",
    axes_labels=labels,
    show_hemisphere_label=True,
    figure_kwargs=fig_kwargs,
    c = cmap)

In [71]:
plot_matchedandtrue(18)

NameError: name 'plot_matchedandtrue' is not defined

In [42]:
oristrue = Orientation.from_euler(np.radians(eulers1[:,:,:]),symmetry=symmetry.Oh)
orismatched = Orientation.from_euler(np.radians(result['orientation'][:,:,0,:]),symmetry=symmetry.Oh)

In [43]:
misangles = np.degrees((oristrue- orismatched).angle.data)
misangles = misangles.flatten()

In [63]:
np.sqrt(np.mean(misangles**2))

40.97825716524681

In [44]:
np.mean(misangles)

3.7052100843214166

In [29]:
np.std(misangles)

14.056356776187195

In [30]:
np.count_nonzero(misangles < 1)/misangles.size

0.0036363636363636364

In [31]:
np.count_nonzero(misangles < 5)/misangles.size

0.02727272727272727

In [34]:
np.count_nonzero(misangles2 < 2)/misangles2.size

0.2

In [33]:
misangles2 = np.delete(misangles, np.where(misangles > 5))

In [35]:
np.sqrt(np.mean(misangles2**2))

3.0990547378362567

In [36]:
np.mean(misangles2)

2.853020201260336

In [37]:
np.std(misangles2)

1.210130571180557

In [38]:
import numpy as np
import scipy.stats as stats

misangles2.sort()
mean = np.mean(misangles2)
std = np.std(misangles2)
pdf = stats.norm.pdf(misangles2, mean, std)
plt.plot(misangles2, pdf) 

In [62]:
misoris = np.zeros((result['orientation'].shape[0],result['orientation'].shape[1]-1,result['orientation'].shape[2]))
for i in range(result['orientation'].shape[1]-1):
    oris0 = Orientation.from_euler(np.radians(result['orientation'][:,i,:,:]),symmetry=symmetry.Oh)
    oris1 = Orientation.from_euler(np.radians(result['orientation'][:,i+1,:,:]),symmetry=symmetry.Oh)
    mis = (oris0[:] - oris1[:])
    misoris[:,i] = np.degrees((mis).angle.data)

In [48]:
misoris[13,:,0]

array([4.74690844, 5.03685504, 6.26239765, 5.19524158, 4.82371699,
       4.81646553, 5.84440027, 5.65674132, 3.604036  , 5.30334922])

In [66]:
misoris2 = misoris[:,:,0].flatten()

In [57]:
np.mean(misoris2)

9.318430068412436

In [41]:
np.std(misoris2)

NameError: name 'misoris2' is not defined

In [60]:
misoris[:,:,0]

array([[ 3.05522291,  7.00207874,  3.57370167, ...,  7.00324446,
         5.3430338 ,  5.11548533],
       [ 6.68368672,  3.89192374,  5.78401525, ...,  6.83002189,
         5.0447401 ,  6.3592596 ],
       [ 5.65571358,  5.38675107,  5.1477056 , ...,  5.60129647,
         4.24118779,  5.22137165],
       ...,
       [ 7.06494051,  3.74344003, 42.33262426, ...,  4.97713246,
         5.59052029, 59.42292021],
       [ 5.09533853,  5.09532819,  5.10153939, ...,  6.64377382,
         9.16672825,  4.22413623],
       [35.27347651, 42.38842371,  5.79342031, ...,  3.5754367 ,
         4.46105559,  5.08582369]])

In [51]:
np.sqrt(np.mean((misoris2 -5)**2))

16.078578144414973

In [67]:
misoris2 = np.delete(misoris2, np.where(np.abs(misoris2-5) > 10))

In [68]:
misoris2.mean()

5.181476876344765

In [70]:
misoris2.size/misoris[:,:,0].size

0.897

In [294]:
import scipy.stats as stats

misoris2.sort()
mean = np.mean(misoris2)
std = np.std(misoris2)
pdf = stats.norm.pdf(misoris2, mean, std)
plt.plot(misoris2, pdf) 

In [129]:
where = np.where(np.abs(misoris[:,:,0]-5) > 2)

In [130]:
where

(array([ 0,  1,  1,  2,  2,  5,  5,  6,  6,  7,  7,  7,  8,  8,  9,  9, 12,
        12, 15, 20, 20, 21, 21, 22, 24, 24, 29, 29, 35, 35, 36, 41, 41, 41,
        48, 51, 51, 53, 53, 56, 56, 58, 58, 64, 65, 66, 66, 70, 78, 78, 80,
        82, 89, 89, 96, 96, 99]),
 array([9, 1, 2, 4, 5, 1, 2, 0, 1, 0, 7, 8, 5, 6, 3, 4, 7, 8, 8, 7, 8, 8,
        9, 9, 0, 9, 1, 2, 4, 5, 7, 2, 3, 9, 8, 1, 2, 3, 4, 8, 9, 3, 4, 3,
        0, 0, 1, 0, 2, 3, 4, 9, 3, 4, 0, 1, 6]))

In [53]:
newbest = np.zeros((result['orientation'].shape[0],result['orientation'].shape[1]),np.int16)

In [54]:
newbest

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

In [128]:
misoris = np.zeros((result['orientation'].shape[0],result['orientation'].shape[1]-1,result['orientation'].shape[2]))
for i in range(result['orientation'].shape[1]-1):
    oris0 = Orientation.from_euler(np.radians(result['orientation'][:,i,:,:]),symmetry=symmetry.Oh)
    oris1 = Orientation.from_euler(np.radians(result['orientation'][:,i+1,:,:]),symmetry=symmetry.Oh)
    mis = (oris0[:] - oris1[:])
    misoris[:,i] = np.degrees((mis).angle.data)

In [132]:
newbest = np.zeros((result['orientation'].shape[0],result['orientation'].shape[1]),np.int16)

for k in range(20):
    newresult= np.zeros((100,11,3))
    for i in range(result['orientation'].shape[0]):
        for j in range(result['orientation'].shape[1]):
            newresult[i,j] = result['orientation'][i,j,newbest[i,j],:]

    misoris = np.zeros((result['orientation'].shape[0],result['orientation'].shape[1]-1))

    for i in range(result['orientation'].shape[1]-1):
        oris0 = Orientation.from_euler(np.radians(newresult[:,i,:]),symmetry=symmetry.Oh)
        oris1 = Orientation.from_euler(np.radians(newresult[:,i+1,:]),symmetry=symmetry.Oh)
        mis = (oris0[:] - oris1[:])
        misoris[:,i] = np.degrees((mis).angle.data)
        # oris0 = oris0.map_into_symmetry_reduced_zone()
        # oris1 = oris1.map_into_symmetry_reduced_zone()
        # misoris[:,i] = np.degrees(oris0.angle_with(oris1).data)

    where = np.where(np.abs(misoris[:,:]-5) > 3)
    for i in range(where[0].size-1):
        if where[0][i] == where[0][i+1]:
            newbest[where[0][i],where[1][i+1]] +=1
        elif where[1][i] == 0:
            newbest[where[0][i],0] +=1
        elif where[1][i] == 9:
            newbest[where[0][i],10] +=1

In [133]:
newbest.nonzero()

(array([ 1,  2,  5,  6,  7,  7,  7,  8,  9, 12, 20, 21, 21, 22, 24, 29, 35,
        41, 41, 41, 53, 56, 56, 58, 65, 66, 70, 78, 82, 89, 96]),
 array([ 2,  5,  2,  1,  0,  7,  8,  6,  4,  8,  8,  9, 10, 10,  0,  2,  5,
         3,  9, 10,  4,  9, 10,  4,  0,  1,  0,  3, 10,  4,  1]))

In [91]:
test =np.squeeze(np.take_along_axis(result['orientation'], newbest[...,np.newaxis,np.newaxis],axis = 2))

In [92]:
np.testing.assert_allclose(test,newresult)

In [74]:
from orix import plot

In [134]:
def plot_matchedandtrue_Oh(num):
    
    
    oritrue = Orientation.from_euler(
        np.radians(eulers1[num,:,:]),
        symmetry=symmetry.Oh
    )
    orimatched = Orientation.from_euler(
    np.radians(result['orientation'][num,:,0,:]),
    symmetry=symmetry.Oh
    )
    orishifted = Orientation.from_euler(
    np.radians(newresult[num,:,:]),
    symmetry=symmetry.Oh
    )
    ckey = plot.IPFColorKeyTSL(oritrue.symmetry)
    rgbz = ckey.orientation2color(oritrue)
    cmap = np.linspace(0, 1, oritrue.size)
    # v = Vector3d(((1, 0, 0), (0, 1, 0), (0, 0, 1)))
    v = Vector3d(((0, 0, 1)))
    oritrue.scatter("ipf", direction=v, c=cmap)
    orimatched.scatter("ipf", direction=v, c=cmap)
    orishifted.scatter("ipf", direction=v, c=cmap)

In [136]:
plot_matchedandtrue_Oh(56)

In [278]:
misoris = np.zeros((result['orientation'].shape[0],result['orientation'].shape[1]-1))
for i in range(result['orientation'].shape[1]-1):
    oris0 = Orientation.from_euler(np.radians(newresult[:,i,:]),symmetry=symmetry.Oh)
    oris1 = Orientation.from_euler(np.radians(newresult[:,i+1,:]),symmetry=symmetry.Oh)
    mis = (oris0[:] - oris1[:])
    misoris[:,i] = np.degrees((mis).angle.data)

In [498]:
where = np.where(np.abs(misoris[:,:]-5) > 5)

In [499]:
where

(array([], dtype=int64), array([], dtype=int64))

In [356]:
newbest.nonzero()

(array([  1,   1,   4,   5,  15,  16,  18,  21,  23,  26,  27,  30,  30,
         33,  36,  39,  39,  39,  43,  43,  44,  50,  51,  52,  57,  63,
         63,  63,  63,  64,  64,  64,  64,  65,  67,  72,  80,  80,  80,
         83,  90,  98,  98, 100, 104, 105, 105, 105, 113, 115, 116, 120,
        123, 124, 124, 126, 130, 135, 136, 139, 146, 147, 152, 152, 152,
        152, 152, 155, 160, 167, 167, 168, 183, 183, 183, 190, 191]),
 array([ 5,  6,  2,  5, 10,  1, 10, 10,  0,  5,  2,  9, 10,  3,  0,  1,  9,
        10,  1,  3,  2,  3,  2,  4,  4,  3,  8,  9, 10,  7,  8,  9, 10,  4,
         8,  1,  4,  6,  7,  0,  1,  9, 10, 10,  3,  3,  7,  8,  5,  0,  0,
         1,  3,  1,  2,  5,  3,  6,  8,  3, 10,  3,  3,  4,  5,  6,  7,  8,
         5,  3,  4,  0,  8,  9, 10,  7,  6]))

In [362]:
newresult= np.zeros((200,11,3))
for i in range(result['orientation'].shape[0]):
    for j in range(result['orientation'].shape[1]):
        newresult[i,j] = result['orientation'][i,j,newbest[i,j],:]

In [97]:
diff_lib.structures

[[Au   0.000000 0.000000 0.000000 1.0000,
  Au   0.000000 0.500000 0.500000 1.0000,
  Au   0.500000 0.000000 0.500000 1.0000,
  Au   0.500000 0.500000 0.000000 1.0000],
 [Ti   0.000000 0.000000 0.000000 1.0000,
  Ti   0.666667 0.333333 0.500000 1.0000,
  Ti   0.333333 0.666667 0.500000 1.0000]]

In [178]:
data1.plot()


In [127]:
diff_lib.keys()

dict_keys(['Au', 'Ti'])

In [104]:
shapetest = result["phase_index"].data.shape
(shapetest[0] * shapetest[1],shapetest [2])

(2200, 10)

In [ ]:
phase_names = 

In [35]:
phase_list= PhaseList(names = phasedict)

In [140]:
xmap.phases.structures

[[Au   0.000000 0.000000 0.000000 1.0000,
  Au   0.000000 0.500000 0.500000 1.0000,
  Au   0.500000 0.000000 0.500000 1.0000,
  Au   0.500000 0.500000 0.000000 1.0000],
 [Ti   0.000000 0.000000 0.000000 1.0000,
  Ti   0.666667 0.333333 0.500000 1.0000,
  Ti   0.333333 0.666667 0.500000 1.0000]]

In [139]:
xmap.phases

Id  Name  Space group  Point group  Proper point group       Color
 0    Au         None         None                None    tab:blue
 1    Ti         None         None                None  tab:orange

In [9]:
from orix import plot

In [138]:
from pyxem.signals.indexation_results import results_dict_to_crystal_map
xmap = test_results_dict_to_crystal_map(result, phasedict, diff_lib)

In [122]:
xmap = test_results_dict_to_crystal_map(result, phasedict)

In [16]:
xmap.phases[0].name = 'au'
xmap.phases[0].space_group = 225

In [113]:
xmap.rotations_per_point

10

In [533]:
(xmap.phase_id == 0).shape

(2200,)

In [528]:
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["au"].point_group, direction=Vector3d.zvector())
rgb_au = ckey_m3m.orientation2color(xmap["au"].orientations)

In [538]:
rgb_au = np.squeeze(rgb_au)

In [539]:
rgb_all = np.zeros((2200, 3))
rgb_all[xmap.phase_id == 0] = rgb_au


xmap.plot(rgb_all)

Specific Tilts

In [128]:
oris0 = Orientation.from_euler(np.radians(eulers1[:,4,:]),symmetry=symmetry.D6h)
oris1 = Orientation.from_euler(np.radians(eulers1[:,5,:]),symmetry=symmetry.D6h)

In [92]:
mis12 = oris0[0] - oris1[0]

In [93]:
mis12

Misorientation (1,) m-3m, m-3m
[[-0.999  -0.0105 -0.0282 -0.0315]]

In [94]:
print(mis12.axis, np.degrees(mis12.angle.data))

Vector3d (1,)
[[0.2409 0.6472 0.7232]] [5.]


In [95]:
r1 = ~oris0[0] * mis12.axis
r1

Vector3d (1,)
[[-0.6288 -0.0073  0.7775]]

In [96]:
r2 = ~oris0[0] * g12.axis
r2

Vector3d (1,)
[[ 0.2749 -0.7573 -0.5924]]

In [98]:
r3 = ~oris0[0] * g12.axis.in_fundamental_sector(symmetry.Oh)
r3

Vector3d (1,)
[[ 0.1386 -0.0626  0.9884]]

In [99]:
fig2 = r1.scatter(c="g", s=200, hemisphere="both", return_figure=True, axes_labels=["x", "y", None])
r2.scatter(c="c", figure=fig2)
r3.scatter(c="y", figure=fig2)

In [56]:
fig3 = g12.axis.scatter(c="r", hemisphere="upper", return_figure=True, axes_labels=["x", "y", None])
ax3 = fig3.axes[0]  # StereographicPlot
ax3.plot(symmetry.Oh.fundamental_sector.edges)
g12.axis.in_fundamental_sector(symmetry.Oh).scatter(c="b", figure=fig3)

In [37]:
oris0 = Orientation.from_euler(np.radians(result['orientation'][:,0,0,:]),symmetry=symmetry.Oh)
oris1 = Orientation.from_euler(np.radians(result['orientation'][:,1,0,:]),symmetry=symmetry.Oh)

In [77]:
oris0 = Orientation.from_euler(np.radians(result['orientation'][6,:,0,:]),symmetry=symmetry.Oh)

In [38]:
mis = (oris0[:] - oris1[:])
# mis = mis.map_into_symmetry_reduced_zone()
mis4 = (oris0[4] - oris1[4])

In [80]:
eulers1[0,0,:]

array([338.        ,  44.67748167,  77.58663764])

In [66]:
np.degrees((mis4).angle.data)

array([55.01641597])

In [154]:
oris0.scatter('rodrigues')

In [159]:
orisomething = Orientation.from_euler(np.radians(result['orientation'][0,:,0,:]),symmetry=symmetry.Oh)

In [160]:
orisomething.scatter('rodrigues')

In [123]:
data1.plot()

In [510]:
newbest[64,9]

1

In [514]:
simulations = diff_lib_au['ti']['simulations']
px = 9
py = 64
n_sol = 0


sim_sol_index = result["template_index"][py, px, n_sol]
mirrored_sol = result["mirrored_template"][py, px, n_sol]
in_plane_angle1 = result["orientation"][py, px, n_sol, 0] #! the first angle is the in plane angle
sim_sol1 = simulations[sim_sol_index]

fig, ax = plt.subplots(ncols=1, figsize=(8, 4))

# solution_vectors_y = Rotation.from_euler(np.deg2rad(solution[:,:,n_sol,...]))*Vector3d.yvector()
# ax[0].imshow(get_ipf_color(solution_vectors_x))
# ax[0].scatter([px], [py], c="black", s=50)
putls.plot_template_over_pattern(data1.inav[px, py].data,
                                 sim_sol1,
                                 ax=ax,
                                 in_plane_angle=in_plane_angle1,
                                 coordinate_system = "cartesian", 
                                 size_factor = 10,
                                 vmax=1, 
                                 max_r = 200,
                                 mirrored_template=mirrored_sol,
                                 find_direct_beam=False,
                                 cmap = "inferno"
                                )


(<AxesSubplot:>,
 <matplotlib.collections.PathCollection at 0x7ff574ba49d0>)

In [509]:
np.log10(50 * 0.01+0.1)

-0.22184874961635637

In [123]:
data1.plot()

In [90]:
ckey = IPFColorKeyTSL(ori.symmetry)
rgbz = ckey.orientation2color(ori)
cmap = np.linspace(0, 1, ori.size)

In [91]:
v = Vector3d(((1, 0, 0), (0, 1, 0), (0, 0, 1)))
ori.scatter("ipf", direction=v, c=cmap)

In [23]:
plot_matchedandtrue(0)